In [113]:
# Group Name: The Awesome Sauce Data Scientist 
# Name: Aryaman Srivastava
# Email: asriva10@stevens.edu
# Country: US
# College: Stevens Institute of Technology
# Specialization: Data Science


# Problem Description:
# ABC Bank wants to sell it's term deposit product to customers and before launching the product they want to develop a model which help them in understanding 
# whether a particular customer will buy their product or not (based on customer's past interaction with bank or other Financial Institution).


In [114]:
import pandas as pd
from sklearn.impute import KNNImputer
from scipy import stats
import numpy as np
df = pd.read_csv("bank-additional-full.csv", delimiter=";")
df2 = pd.read_csv("bank-full.csv", delimiter=";")
df3 = pd.read_csv("bank.csv", delimiter=";")
df4 = pd.read_csv("bank-additional.csv", delimiter=";")

merged_additional_df = pd.concat([df4, df], ignore_index=True)
merged_bank_df = pd.concat([df3, df2], ignore_index=True)

# method 1 for NA values: replace all NA values with mean
merged_additional_df["age"] = merged_additional_df["age"].fillna(merged_additional_df["age"].mean())
merged_additional_df["campaign"] = merged_additional_df["campaign"].fillna(merged_additional_df["campaign"].mean())
merged_additional_df["pdays"] = merged_additional_df["pdays"].fillna(merged_additional_df["pdays"].mean())
merged_additional_df["previous"] = merged_additional_df["previous"].fillna(merged_additional_df["previous"].mean())
merged_additional_df["emp.var.rate"] = merged_additional_df["emp.var.rate"].fillna(merged_additional_df["emp.var.rate"].mean())
merged_additional_df["cons.price.idx"] = merged_additional_df["cons.price.idx"].fillna(merged_additional_df["cons.price.idx"].mean())
merged_additional_df["cons.conf.idx"] = merged_additional_df["cons.conf.idx"].fillna(merged_additional_df["cons.conf.idx"].mean())
merged_additional_df["euribor3m"] = merged_additional_df["euribor3m"].fillna(merged_additional_df["euribor3m"].mean())
merged_additional_df["nr.employed"] = merged_additional_df["nr.employed"].fillna(merged_additional_df["nr.employed"].mean())

merged_bank_df["age"] = merged_bank_df["age"].fillna(merged_bank_df["age"].mean())
merged_bank_df["balance"] = merged_bank_df["balance"].fillna(merged_bank_df["balance"].mean())
merged_bank_df["duration"] = merged_bank_df["duration"].fillna(merged_bank_df["duration"].mean())
merged_bank_df["campaign"] = merged_bank_df["campaign"].fillna(merged_bank_df["campaign"].mean())
merged_bank_df["pdays"] = merged_bank_df["pdays"].fillna(merged_bank_df["pdays"].mean())
merged_bank_df["previous"] = merged_bank_df["previous"].fillna(merged_bank_df["previous"].mean())

In [115]:
# method 2 for NA values: use a simple model to predict NA values
numerical_cols = [
    "age",
    "campaign",
    "pdays",
    "previous",
    "emp.var.rate",
    "cons.price.idx",
    "cons.conf.idx",
    "euribor3m",
    "nr.employed",
]
numerical_df = merged_additional_df[numerical_cols]
categorical_cols = merged_additional_df.columns.difference(numerical_cols)
imputer = KNNImputer(n_neighbors=2)
numerical_df = pd.DataFrame(
    imputer.fit_transform(numerical_df),
    columns=numerical_cols
)
merged_additional_df = pd.concat([numerical_df, merged_additional_df[categorical_cols]], axis=1)
numerical_cols = ["age", "balance", "duration", "campaign", "pdays", "previous"]
numerical_df = merged_bank_df[numerical_cols]
categorical_cols = merged_bank_df.columns.difference(numerical_cols)
imputer = KNNImputer(n_neighbors=2)
numerical_df = pd.DataFrame(
    imputer.fit_transform(numerical_df),
    columns=numerical_cols,
)
merged_bank_df = pd.concat([numerical_df, merged_bank_df[categorical_cols]], axis=1)


In [116]:
# method 1 for removing outliers using IQR rule

numerical_cols = [
    "age",
    "campaign",
    "pdays",
    "previous",
    "emp.var.rate",
    "cons.price.idx",
    "cons.conf.idx",
    "euribor3m",
    "nr.employed",
]
numerical_df = merged_additional_df[numerical_cols]
categorical_cols = merged_additional_df.columns.difference(numerical_cols)
Q1 = numerical_df[numerical_cols].quantile(0.25)
Q3 = numerical_df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1
numerical_df = numerical_df[~(
        (numerical_df[numerical_cols] < (Q1 - 1.5 * IQR))
        | (numerical_df[numerical_cols] > (Q3 + 1.5 * IQR))
    ).any(axis=1)]
merged_additional_df = merged_additional_df.loc[numerical_df.index]
merged_additional_df = pd.concat([numerical_df, merged_additional_df[categorical_cols]], axis=1)

numerical_cols = ["age", "balance", "duration", "campaign", "pdays", "previous"]
numerical_df = merged_bank_df[numerical_cols]
categorical_cols = merged_bank_df.columns.difference(numerical_cols)
Q1 = numerical_df[numerical_cols].quantile(0.25)
Q3 = numerical_df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1
numerical_df = numerical_df[
    ~(
        (numerical_df[numerical_cols] < (Q1 - 1.5 * IQR))
        | (numerical_df[numerical_cols] > (Q3 + 1.5 * IQR))
    ).any(axis=1)
]
merged_bank_df = merged_bank_df.loc[numerical_df.index]
merged_bank_df = pd.concat([numerical_df, merged_bank_df[categorical_cols]], axis=1)

In [117]:
# method 2 for removing outliers using zscores
numerical_cols = [
    "age",
    "campaign",
    "pdays",
    "previous",
    "emp.var.rate",
    "cons.price.idx",
    "cons.conf.idx",
    "euribor3m",
    "nr.employed",
]
numerical_df = merged_additional_df[numerical_cols]
categorical_cols = merged_additional_df.columns.difference(numerical_cols)

# Calculate Z-scores and filter out rows with outliers
z_scores = np.abs(stats.zscore(numerical_df))
numerical_df = numerical_df[(z_scores < 3).all(axis=1)]

# Filter categorical columns to match the filtered numerical rows
merged_additional_df = merged_additional_df.loc[numerical_df.index]

# Combine the filtered numerical and categorical data
merged_additional_df = pd.concat(
    [numerical_df, merged_additional_df[categorical_cols]], axis=1
)

numerical_cols = ["age", "balance", "duration", "campaign", "pdays", "previous"]
numerical_df = merged_bank_df[numerical_cols]
categorical_cols = merged_bank_df.columns.difference(numerical_cols)
z_scores = np.abs(stats.zscore(numerical_df))
numerical_df = numerical_df[(z_scores < 3).all(axis=1)]
merged_bank_df = merged_bank_df.loc[numerical_df.index]
merged_bank_df = pd.concat(
    [numerical_df, merged_bank_df[categorical_cols]], axis=1
)